# Management policies

This notebook shows how a host should query a manager to discover its policy toward managing different categories of asset using the `managementPolicy` API, both for reading existing data and publishing new data.

## Setup

See "Hello OpenAssetIO" notebook for details on how to bootstrap OpenAssetIO.

In [1]:
from resources import helpers


manager, context = helpers.bootstrap("resources/management_policies/openassetio_config.toml")

## Setting the scene

Our DCC (Digital Content Creation) tool wishes to load a 3D model, render it to a 2D image, then publish that render.

Managers have varying capabilities, and we want to adjust the user workflow, both in terms of UI presented and the entity queries performed, to suit the information the manager is capable of providing and persisting.

## Determining resolvable geometry properties

Say a user wishes to load the geometry, then a common issue with geometry formats is knowing which axis is intended to be used as the "up" direction. 

Ideally the manager would tell us which axis to consider as "up". However, we cannot assume the manager is capable of this. If it can't, we want to display a variation of the UI that additionally requires the user to specify the base orientation.

We can decide ahead of time which UI should be shown by querying the manager's policy for this type of entity. This will tell us which traits of the entity that the manager can `resolve`.

The MediaCreation package has an appropriate Specification for describing 3D geometry, so let's use that.

In [2]:
from openassetio_mediacreation.specifications.threeDimensional import \
    SceneGeometryResourceSpecification


helpers.display_result(SceneGeometryResourceSpecification.kTraitSet)

> **Result:**
> `{'openassetio-mediacreation:threeDimensional.Geometry', 'openassetio-mediacreation:threeDimensional.Spatial', 'openassetio-mediacreation:usage.Entity', 'openassetio-mediacreation:content.LocatableContent'}`

If we look at the definition of these traits (see `traits.yml`)
 - Only `LocatableContent` and `Spatial` traits have potentially resolvable properties. The other traits are purely to aid in classification. 
 - To get the "up" axis of some geometry, we would need to `resolve` the `Spatial` trait and retrieve its `upAxis` property.

Now lets get the manager's policy for resolving existing entities of this category.

In [3]:
from openassetio.access import PolicyAccess


[policy] = manager.managementPolicy(
    [SceneGeometryResourceSpecification.kTraitSet], PolicyAccess.kRead, context)

helpers.display_result(policy.traitSet())

> **Result:**
> `{'openassetio-mediacreation:content.LocatableContent', 'openassetio-mediacreation:managementPolicy.Managed'}`

The returned policy object is a bundle of traits, much like the result of `resolve`, but with a significantly different meaning.

The policy trait set is a combination of (1) policy-specific traits (those marked with a `usage` of `managementPolicy` in their definition), and (2) the subset of the requested traits that can be resolved.

In this case, the `Managed` policy-specific trait is imbued, and (only) the `LocatableContent` entity trait (from the `SceneGeometryResourceSpecification`'s trait set) is imbued.

The good news is that, since the `Managed` policy trait is imbued, we know that this type of entity can be queried.

However, note the absence of the `Spatial` trait in the response. This means that, unfortunately, the manager is not capable of resolving that trait. Be aware, though, that the `Spatial` trait is still part of the entity's trait set and still affects its categorization, even if the manager is not capable of providing data for it.

The Specification and Trait view classes provide convenient utility methods to safely inspect trait data for the presence of a trait:

In [4]:
from openassetio_mediacreation.traits.managementPolicy import ManagedTrait
from openassetio_mediacreation.traits.threeDimensional import SpatialTrait

# Check if policy contains the `ManagedTrait`
assert ManagedTrait.isImbuedTo(policy) == True

# Check if policy contains the `SpatialTrait`
assert SpatialTrait(policy).isImbued() == False
# or
assert SceneGeometryResourceSpecification(policy).spatialTrait().isImbued() == False


Now that we have determined what can be resolved, a host author can use this information to tailor their UI accordingly, prompting the user for information the manager cannot provide, and alleviating the user from having to fill in any redundant information that the manager can provide.

## Determining requirements for publishing

For publishing our rendered image, we will need to inform the manager of the image entity's trait set. We make use of the MediaCreation `PlanarBitmapImageResourceSpecification` and, since our application uses OpenColorIO to manage the colour space of renders, we augment the specification's trait set with an optional `OCIOColorManaged` trait, to signal that our images have this additional quality.

In [5]:
from openassetio_mediacreation.specifications.twoDimensional import \
    PlanarBitmapImageResourceSpecification
from openassetio_mediacreation.traits.color import OCIOColorManagedTrait


image_trait_set = PlanarBitmapImageResourceSpecification.kTraitSet | {OCIOColorManagedTrait.kId}

helpers.display_result(image_trait_set)


> **Result:**
> `{'openassetio-mediacreation:color.OCIOColorManaged', 'openassetio-mediacreation:twoDimensional.PixelBased', 'openassetio-mediacreation:content.LocatableContent', 'openassetio-mediacreation:twoDimensional.Planar', 'openassetio-mediacreation:twoDimensional.Image', 'openassetio-mediacreation:usage.Entity'}`

When publishing an entity (i.e. using the `preflight` and `register` API methods), the published data should be imbued with _all_ the traits from an entity's trait set (plus any optional traits you wish the manager to consider), in order for the manager to properly classify the entity. 

However, traits can also carry data in the form of trait _properties_. If a manager accepts a given trait set for publishing, that doesn't necessarily mean the manager can persist all the _properties_ associated with those traits.

So we have two questions. We want to know whether the manager is interested in entities of this type at all. If so, we want to know the subset of traits whose properties can be persisted by the manager. 

We can answer these questions by calling `managementPolicy` with the `kWrite` access mode:

In [6]:
from openassetio.access import PolicyAccess


[write_policy] = manager.managementPolicy([image_trait_set], PolicyAccess.kWrite, context)

helpers.display_result(write_policy.traitSet())

> **Result:**
> `{'openassetio-mediacreation:content.LocatableContent', 'openassetio-mediacreation:color.OCIOColorManaged', 'openassetio-mediacreation:twoDimensional.PixelBased', 'openassetio-mediacreation:managementPolicy.Managed'}`

Once again we see the `Managed` trait, indicating that entities of this kind can be published. In addition, we see that `PixelBased`, `LocatableContent` and `OCIOColorManaged` are traits with associated properties that can be persisted by the manager.

Again, these are traits that _can be_ persisted by the manager. However, some properties may be _required_ for publishing to succeed. For this, we can again check the manager's policy, where we wish to ask the manager for the `kRequired` subset of an entity's traits:

In [7]:
[required_policy] = manager.managementPolicy([image_trait_set], PolicyAccess.kRequired, context)

helpers.display_result(required_policy.traitSet())

> **Result:**
> `{'openassetio-mediacreation:color.OCIOColorManaged', 'openassetio-mediacreation:managementPolicy.Managed', 'openassetio-mediacreation:content.LocatableContent'}`

We can see that the required traits are `LocatableContent` and `OCIOColorManaged` (i.e. the properties of `PixelBased` are optional). 

Note that even within a single given trait, not all properties are necessarily required. Consult the documentation for each specific trait to learn which properties may be optional.

Generally, it is a good idea to imbue all the information you have, as the manager is in the best place to decide what is does or doesn't want to persist.

Before performing a render, we should check if the manager has any values that it wants to dictate to the host ahead of time. 

In [8]:
[manager_driven_policy] = manager.managementPolicy(
    [image_trait_set], PolicyAccess.kManagerDriven, context)

helpers.display_result(manager_driven_policy.traitSet())

> **Result:**
> `{'openassetio-mediacreation:content.LocatableContent', 'openassetio-mediacreation:managementPolicy.Managed'}`

This particular manager wishes the output to be written to a specific destination on the file system (since the `LocatableContent` trait is in the result). However, the manager has no opinion on which colour space to use (since `OCIOColorManaged` is not in the result), so the host is free to use whichever it wants, as long as the colour space is published with the rest of the data (since it is `kRequired`, see above).

We now know that when we start publishing an entity of this kind, we can (indeed, should) take the working reference returned from `preflight`, and `resolve` it for the `LocatableContent` trait in `kManagerDriven` access mode, in order to get the file save path. An example of this workflow is given below.

## Usage in publishing

Now that we have the pieces, lets sketch out what a publishing workflow could look like. In particular, note the `resolve` of the working reference with `ResolveAccess.kManagerDriven`

In [9]:
from openassetio.access import PublishingAccess, ResolveAccess

from openassetio_mediacreation.traits.content import LocatableContentTrait
from openassetio_mediacreation.traits.twoDimensional import PixelBasedTrait
from openassetio_mediacreation.traits.color import OCIOColorManagedTrait


image_trait_set = PlanarBitmapImageResourceSpecification.kTraitSet | {OCIOColorManagedTrait.kId}

[write_policy] = manager.managementPolicy([image_trait_set], PolicyAccess.kWrite, context)
[required_policy] = manager.managementPolicy([image_trait_set], PolicyAccess.kRequired, context)
[manager_driven_policy] = manager.managementPolicy(
    [image_trait_set], PolicyAccess.kManagerDriven, context)

# In practice, host applications should gracefully handle the policy of
# the manager (e.g. by allowing the user to select a file save location
# if the manager cannot provide one). Here, we just fail if the policy
# is not what we expect.

if not ManagedTrait.isImbuedTo(write_policy):
    raise RuntimeError("Manager does not support publishing image types")

if not LocatableContentTrait.isImbuedTo(write_policy):
    raise RuntimeError("Manager cannot store file save location")

if PixelBasedTrait.isImbuedTo(required_policy):
    raise RuntimeError("Manager requires image metadata that cannot be provided")

if OCIOColorManagedTrait.isImbuedTo(manager_driven_policy):
    raise RuntimeError("Manager wants to dictate colour space but host does not support this")

if not LocatableContentTrait.isImbuedTo(manager_driven_policy):
    raise RuntimeError("Manager cannot provide a location to save to")

# Create an image specification, and additionally imbue the optional
# OCIO trait.
image_spec = PlanarBitmapImageResourceSpecification.create()
image_traits_data = image_spec.traitsData()
ocio_trait = OCIOColorManagedTrait(image_traits_data)
ocio_trait.imbue()
# We know the colour space we're going to use ahead of time, so we should
# pre-fill this data ready for `preflight` to begin the (relatively
# long-running) publishing process.
ocio_trait.setColorspace("sRGB")

# Let's pretend the user has entered the following entity reference in a
# publishing widget.
ref_str_from_widget = "bal:///project/renders/myrender"

initial_ref = manager.createEntityReference(ref_str_from_widget)

# Initiate publishing, preflighting all available up-front data to obtain
# a working reference that we will `resolve` from and `register` to, below.
working_ref = manager.preflight(
    initial_ref, image_traits_data, PublishingAccess.kWrite, context)

# Get the data that is `kManagerDriven`, i.e. dictated by the manager.
manager_driven_data = manager.resolve(
    working_ref, {LocatableContentTrait.kId}, ResolveAccess.kManagerDriven, context)

manager_driven_locatable_content_trait = LocatableContentTrait(manager_driven_data)
save_file_path = manager_driven_locatable_content_trait.getLocation()
save_file_type = manager_driven_locatable_content_trait.getMimeType(defaultValue="image/x-exr")

print(f"Rendering to '{save_file_path}' with type '{save_file_type}'")

# [... do the render ...]

# Set the final save location and type in the data to publish. This should
# be the same as the location `resolve`d above.
locatable_content_trait = LocatableContentTrait(image_traits_data)
locatable_content_trait.setLocation(save_file_path)
locatable_content_trait.setMimeType(save_file_type)

final_ref = manager.register(working_ref, image_traits_data, PublishingAccess.kWrite, context)

helpers.display_result(final_ref)

Rendering to 'file:///mnt/staging/renders/myrender.exr' with type 'image/x-exr'


> **Result:**
> `bal:///project/renders/myrender?v=2`